In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import os
from pathlib import Path
import matplotlib.pyplot as plt
import re
from textwrap import wrap

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (8,5)

In [ ]:
!pip install kagglehub


In [ ]:
import kagglehub

path = kagglehub.dataset_download("kanchana1990/ai-and-ml-job-listings-usa")

print("Ruta local del dataset:", path)


## Paso 1. Verificar que el dataset es tabular

In [ ]:
path = Path(path)
print("Ruta local del dataset:", path)

all_files = [p for p in path.rglob("*") if p.is_file()]
print(f"Archivos encontrados ({len(all_files)}):")
for p in all_files[:15]:
    print(" -", p.name)

img_exts = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}
tabular_exts = {".csv", ".tsv", ".xlsx", ".xls", ".parquet"}

has_images = any(p.suffix.lower() in img_exts for p in all_files)
tabular_files = [p for p in all_files if p.suffix.lower() in tabular_exts]

if has_images and not tabular_files:
    raise ValueError("El dataset parece tener imágenes y no archivos tabulares. Escoge otro dataset solo de datos.")
elif not tabular_files:
    raise ValueError("No se detectaron archivos tabulares (.csv/.tsv/.xlsx/.xls/.parquet). Revisa el contenido del dataset.")
else:
    print("\nOK: Se detectaron archivos tabulares:")
    for p in tabular_files:
        print(" -", p.name)

def file_priority(p: Path) -> int:
    ext = p.suffix.lower()
    if ext == ".parquet": return 0
    if ext in [".csv", ".tsv"]: return 1
    if ext in [".xlsx", ".xls"]: return 2
    return 9

tabular_files_sorted = sorted(tabular_files, key=lambda p: (file_priority(p), -p.stat().st_size))
DATA_FILE = tabular_files_sorted[0]
print(f"\nArchivo principal elegido: {DATA_FILE.name} (tamaño ~ {DATA_FILE.stat().st_size/1e6:.2f} MB)")

## Paso 2. Cargar el CSV

In [ ]:
DATA_FILE = next(p for p in Path(path).rglob("ai_ml_jobs_linkedin.csv"))
df = pd.read_csv(DATA_FILE, low_memory=False)
print(f"Cargado: {df.shape[0]:,} filas x {df.shape[1]:,} columnas")
display(df.head(5))

## Paso 3. Chequeos básicos (nulos, duplicados, cardinalidad)

In [ ]:
print("\nTipos originales:")
print(df.dtypes)

print("\n% de nulos por columna (Top 20):")
nulls = df.isna().mean().sort_values(ascending=False)
display(nulls.head(20).to_frame("% nulos"))

dup_count = df.duplicated().sum()
print(f"\nDuplicados exactos: {dup_count}")

card = df.nunique(dropna=True).sort_values(ascending=False)
print("\nCardinalidad por columna (Top 20):")
display(card.head(20).to_frame("nunique"))

## Paso 4. Tipificar columnas (num/cat/fecha)

In [ ]:
if "df_clean" not in locals():
    df_clean = df.copy()

for col in df_clean.select_dtypes(include=["object"]).columns:
    df_clean[col] = df_clean[col].astype(str).str.strip().replace({"nan": np.nan})

if "publishedAt" in df_clean.columns:
    df_clean["publishedAt"] = pd.to_datetime(df_clean["publishedAt"], errors="coerce", utc=True).dt.tz_localize(None)

def parse_applicants(s):
    if pd.isna(s):
        return np.nan
    nums = re.findall(r"\d+", str(s))
    if not nums:
        return np.nan
    return float(max(int(n) for n in nums))

if "applicationsCount_num" not in df_clean.columns:
    if "applicationsCount" in df_clean.columns:
        df_clean["applicationsCount_num"] = df_clean["applicationsCount"].apply(parse_applicants)
    else:
        pass

if "applicationsCount_num" in df_clean.columns and "is_top200" not in df_clean.columns:
    df_clean["is_top200"] = (df_clean["applicationsCount_num"] >= 200).astype("Int64")

if "applicationsCount_num" in df_clean.columns and "publishedAt" in df_clean.columns and "apps_per_day" not in df_clean.columns:
    today = pd.Timestamp.today().normalize()
    days = (today - df_clean["publishedAt"]).dt.days
    days = days.clip(lower=1)
    df_clean["apps_per_day"] = df_clean["applicationsCount_num"] / days
    df_clean.loc[df_clean["apps_per_day"] > 1000, "apps_per_day"] = np.nan

if "title" in df_clean.columns:
    df_clean["title_len"] = df_clean["title"].astype(str).str.len()
if "description" in df_clean.columns:
    df_clean["desc_len"] = df_clean["description"].astype(str).str.len()
if "location" in df_clean.columns and "state" not in df_clean.columns:
    df_clean["state"] = (
        df_clean["location"].astype(str).str.extract(r",\s*([A-Z]{2})", expand=False)
    )
if "sector" in df_clean.columns and "sector_main" not in df_clean.columns:
    df_clean["sector_main"] = df_clean["sector"].astype(str).str.split(",").str[0].str.strip()
if "location" in df_clean.columns and "location_clean" not in df_clean.columns:
    bad_locs = {"United States", "New York, United States"}
    df_clean["location_clean"] = df_clean["location"].where(~df_clean["location"].isin(bad_locs), np.nan)

for col in df_clean.select_dtypes(include=["object"]).columns:
    nunq = df_clean[col].nunique(dropna=True)
    if 2 <= nunq <= 50:
        df_clean[col] = df_clean[col].astype("category")

num_base = [c for c in ["applicationsCount_num","apps_per_day","title_len","desc_len"] if c in df_clean.columns]
for c in num_base:
    s = df_clean[c]
    if s.notna().any() and not (s.dropna() < 0).any():
        df_clean[f"log1p_{c}"] = np.log1p(s)

for c in ["title_len","desc_len"]:
    if c in df_clean.columns and f"{c}_win" not in df_clean.columns:
        hi = df_clean[c].quantile(0.99)
        df_clean[f"{c}_win"] = np.minimum(df_clean[c], hi)

created = [c for c in ["applicationsCount_num","is_top200","apps_per_day",
                       "title_len","desc_len","title_len_win","desc_len_win",
                       "log1p_applicationsCount_num","log1p_apps_per_day",
                       "log1p_title_len","log1p_desc_len",
                       "state","sector_main","location_clean"]
           if c in df_clean.columns]
print("Features creadas/actualizadas:", created)
print("Tipos (resumen):")
print(df_clean[created].dtypes if created else "—")

## PASO 5. Distribuciones

In [ ]:
num_cols = ["applicationsCount_num", "title_len", "desc_len"]
num_cols = [c for c in num_cols if c in df_clean.columns]

cat_cols = []
for c in ["contractType","experienceLevel","workType","state","sector"]:
    if c in df_clean.columns:
        cat_cols.append(c)

print(f"Numéricas: {num_cols}")
print(f"Categóricas: {cat_cols}")

for col in num_cols:
    ax = df_clean[col].plot(kind="hist", bins=40, title=f"Histograma: {col}")
    ax.set_xlabel(col); ax.set_ylabel("Frecuencia")
    plt.show()

for col in cat_cols:
    vc = df_clean[col].value_counts(dropna=False).head(20)
    ax = vc.plot(kind="bar", title=f"{col} (Top 20)")
    ax.set_xlabel(col); ax.set_ylabel("Frecuencia")
    plt.show()

if "publishedAt" in df_clean.columns:
    s = (
        df_clean.set_index("publishedAt")
                .sort_index()
                .resample("W")
                .size()
    )
    ax = s.plot(kind="line", title="Publicaciones por semana")
    ax.set_xlabel("Fecha"); ax.set_ylabel("Conteo")
    plt.show()

## PASO 6. Univariable

In [ ]:
if num_cols:
    desc_num = df_clean[num_cols].describe().T
    desc_num["missing_%"] = 100*(1 - df_clean[num_cols].notna().mean())
    desc_num["skew"] = df_clean[num_cols].skew(numeric_only=True)
    desc_num["kurt"] = df_clean[num_cols].kurt(numeric_only=True)
    print("\nEstadísticos numéricos:")
    display(desc_num)

for col in cat_cols:
    print(f"\nTop categorías: {col}")
    display(df_clean[col].value_counts(dropna=False))

## PASO 7. Multivariable

In [ ]:
TARGET = "applicationsCount_num"
TOP_K = 20
MIN_COUNT = 10
WRAP_WIDTH = 26
MAX_LINES = 2

def wrap_or_trim(labels, width=26, max_lines=2):
    out = []
    for lab in labels:
        s = str(lab)
        if "," in s and len(s) > width:
            parts = [p.strip() for p in s.split(",")]
            s = ", ".join(parts[:2])
        w = wrap(s, width=width)
        if len(w) > max_lines:
            w = w[:max_lines-1] + [w[max_lines-1][:max(0, width-3)] + "..."]
        out.append("\n".join(w))
    return out

if "sector" in df_clean.columns and "sector_main" not in df_clean.columns:
    df_clean["sector_main"] = df_clean["sector"].astype(str).str.split(",").str[0].str.strip()

bad_locs = {"United States", "New York, United States"}
if "location" in df_clean.columns and "location_clean" not in df_clean.columns:
    df_clean["location_clean"] = df_clean["location"].where(~df_clean["location"].isin(bad_locs), np.nan)

cat_cols = [c for c in [
    "contractType","experienceLevel","workType",
    "state","sector_main","sector",
    "companyName","title","location_clean","location"
] if c in df_clean.columns and df_clean[c].dtype.name in ["object","category"]]

def barras_medianas_sin_error(col, top_k=TOP_K, min_count=MIN_COUNT):
    agg = (
        df_clean.groupby(col, observed=True)[TARGET]
        .agg(n="count", mediana="median")
        .query("n >= @min_count")
        .sort_values("mediana", ascending=False)
        .head(top_k)
    )
    if agg.empty: return

    x = agg["mediana"].values
    ylabels = wrap_or_trim(agg.index.tolist(), WRAP_WIDTH, MAX_LINES)

    plt.figure(figsize=(12, 6))
    plt.barh(range(len(x)), x)
    plt.yticks(range(len(x)), ylabels)
    plt.gca().invert_yaxis()

    max_x = x.max()
    plt.xlim(0, max_x * 1.1)
    plt.xlabel(f"Mediana de {TARGET}")
    plt.title(f"Mediana de {TARGET} por {col} (Top {len(x)}, min {min_count} filas)")
    plt.tight_layout(); plt.show()

    print(f"\n{col} — Top {len(agg)} por mediana (min {min_count} filas)")
    display(agg)

def barras_prop_tope_sin_error(col, threshold=200, top_k=TOP_K, min_count=MIN_COUNT):
    df_aux = df_clean[[col, TARGET]].dropna(subset=[col])
    if df_aux.empty: return
    df_aux = df_aux.assign(al_tope = df_aux[TARGET] >= threshold)
    agg = (
        df_aux.groupby(col, observed=True)
        .agg(n=(TARGET, "count"), prop_top=("al_tope", "mean"))
        .query("n >= @min_count")
        .sort_values("prop_top", ascending=False)
        .head(top_k)
    )
    if agg.empty: return

    x = (agg["prop_top"] * 100).values
    ylabels = wrap_or_trim(agg.index.tolist(), WRAP_WIDTH, MAX_LINES)

    plt.figure(figsize=(12, 5.5))
    plt.barh(range(len(x)), x)
    plt.yticks(range(len(x)), ylabels)
    plt.gca().invert_yaxis()

    max_x = x.max()
    pad = max(1e-9, max_x * 0.02)
    plt.xlim(0, max_x * 1.1)
    for i, v in enumerate(x):
        plt.text(v + pad, i, f"{v:.0f}%", va="center", ha="left", fontsize=10)

    plt.xlabel(f"% de ofertas con ≥{threshold} aplicaciones")
    plt.title(f"% al tope por {col} (Top {len(x)}, min {min_count} filas)")
    plt.tight_layout(); plt.show()

    print(f"\n{col} — % al tope (≥{threshold})")
    display(agg)

for col in cat_cols:
    print("\n" + "="*80)
    print(f"Análisis categórica: {col}")
    barras_medianas_sin_error(col)
    barras_prop_tope_sin_error(col)

In [57]:

if "apps_per_day" not in df_clean.columns and "publishedAt" in df_clean.columns:
    today = pd.Timestamp.today().normalize()
    days = (today - df_clean["publishedAt"]).dt.days
    days = days.clip(lower=1)
    df_clean["apps_per_day"] = df_clean["applicationsCount_num"] / days
    df_clean.loc[df_clean["apps_per_day"] > 1000, "apps_per_day"] = np.nan


## PASO 8. Outliers

In [ ]:

num_cols = [c for c in ["applicationsCount_num", "title_len", "desc_len", "apps_per_day"]
            if c in df_clean.columns]

P_LOW, P_HIGH = 0.01, 0.99
BINS = 40

def iqr_bounds(s, k=1.5):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

def _clip_for_plot(s: pd.Series, low=P_LOW, high=P_HIGH):
    s = s.dropna()
    if s.empty:
        return s, None, None
    lo, hi = s.quantile(low), s.quantile(high)
    return s.clip(lo, hi), lo, hi

def hist_safe(s: pd.Series, name: str, bins=BINS):
    s = s.dropna()
    if s.empty:
        return
    s_clip, lo, hi = _clip_for_plot(s)

    plt.figure(figsize=(6,4))
    plt.hist(s_clip.values, bins=bins)
    plt.title(f"Histograma (cap {int(P_LOW*100)}–{int(P_HIGH*100)}%): {name}")
    plt.xlabel(name); plt.ylabel("Frecuencia")

    if s.skew() > 1:
        plt.figure(figsize=(6,4))
        plt.hist(np.log1p(s.values), bins=bins)
        plt.title(f"Histograma: log1p({name})")
        plt.xlabel(f"log1p({name})"); plt.ylabel("Frecuencia")
        plt.tight_layout(); plt.show()

def box_safe(s: pd.Series, name: str):
    s = s.dropna()
    if s.empty:
        return
    s_clip, lo, hi = _clip_for_plot(s)
    plt.figure(figsize=(6,4))
    plt.boxplot(s_clip.values, showfliers=False)
    plt.title(f"Boxplot (sin fliers, cap {int(P_LOW*100)}–{int(P_HIGH*100)}%): {name}")
    plt.ylabel(name)
    plt.tight_layout(); plt.show()


rows = []
for col in num_cols:
    s = df_clean[col].dropna()
    if s.empty:
        continue
    low, high = iqr_bounds(s, 1.5)
    iqr_flags = (s < low) | (s > high)
    z = (s - s.mean())/s.std(ddof=0)
    z_flags = z.abs() > 3
    rows.append({
        "columna": col, "n": int(s.size),
        "%_outliers_IQR": round(iqr_flags.mean()*100, 2),
        "%_outliers_Z>3": round(z_flags.mean()*100, 2),
        "lim_inf_IQR": low, "p25": s.quantile(.25), "mediana": s.median(),
        "p75": s.quantile(.75), "lim_sup_IQR": high,
        "min": s.min(), "max": s.max(), "skew": round(s.skew(), 3)
    })

out_df = pd.DataFrame(rows).sort_values("%_outliers_IQR", ascending=False)
print("\nResumen de outliers (IQR y Z>3):")
display(out_df)

for col in num_cols:
    s = df_clean[col]
    box_safe(s, col)
    hist_safe(s, col)

##  PASO 9. Correlaciones

In [ ]:
RAW_COLS = [c for c in ["applicationsCount_num","apps_per_day","title_len","desc_len"] if c in df_clean.columns]
USE_LOG_ON_THE_FLY = True
SCATTER_SAMPLE_MAX = 10000
P_LOW, P_HIGH = 0.01, 0.99
POINT_SIZE = 8
ALPHA = 0.3

def build_log_view(df, cols):
    """Devuelve un DataFrame con log1p de cols si todas existen y son >=0 (al menos no-negativas).
       No modifica df_clean: es una vista temporal."""
    if not cols:
        return None, []
    ok = []
    for c in cols:
        s = df[c].dropna()
        if s.empty:
            continue
        if (s < 0).any():
            continue
        ok.append(c)
    if not ok:
        return None, []
    df_log = pd.DataFrame({f"log1p_{c}": np.log1p(df[c]) for c in ok})
    return df_log, [f"log1p_{c}" for c in ok]

def corr_and_top_pairs(df_num, name):
    cols = df_num.columns.tolist()
    if len(cols) < 2:
        print(f"\n[{name}] No hay suficientes columnas para correlaciones.")
        return None, None
    print(f"\n=== Correlaciones ({name}) ===")
    corr_s = df_num.corr(method="spearman")
    corr_k = df_num.corr(method="kendall")
    print("Spearman:"); display(corr_s)
    print("Kendall:");  display(corr_k)

    pair_scores = []
    for a, b in combinations(cols, 2):
        val = abs(corr_s.loc[a, b])
        if pd.notna(val):
            pair_scores.append(((a, b), val))
    pair_scores.sort(key=lambda x: x[1], reverse=True)
    top_pairs = pair_scores[:10]
    if top_pairs:
        print("Top pares (|Spearman|):")
        display(pd.Series({f"{a} ~ {b}": v for (a, b), v in top_pairs}))
    else:
        print("Sin pares válidos.")
    return corr_s, top_pairs

def scatter_robusto(df, x, y, title=None):
    s = df[[x, y]].dropna()
    if s.empty:
        return
    if len(s) > SCATTER_SAMPLE_MAX:
        s = s.sample(SCATTER_SAMPLE_MAX, random_state=42)

    xlo, xhi = s[x].quantile(P_LOW), s[x].quantile(P_HIGH)
    ylo, yhi = s[y].quantile(P_LOW), s[y].quantile(P_HIGH)
    sx = s[x].clip(xlo, xhi)
    sy = s[y].clip(ylo, yhi)

    plt.figure(figsize=(6,5))
    plt.scatter(sx, sy, s=POINT_SIZE, alpha=ALPHA)
    plt.xlabel(x); plt.ylabel(y)
    plt.title(title if title else f"{x} vs {y} (cap {int(P_LOW*100)}–{int(P_HIGH*100)}%)")
    plt.tight_layout(); plt.show()

if RAW_COLS:
    df_raw = df_clean[RAW_COLS]
    _, top_pairs_raw = corr_and_top_pairs(df_raw, "RAW")
else:
    top_pairs_raw = None
    print("No hay columnas numéricas RAW.")

top_pairs_log = None
if USE_LOG_ON_THE_FLY:
    df_log, LOG_COLS = build_log_view(df_clean, RAW_COLS)
    if df_log is not None and len(LOG_COLS) >= 2:
        _, top_pairs_log = corr_and_top_pairs(df_log, "LOG1P (al vuelo)")
    else:
        print("\n[LOG1P] No se generaron suficientes columnas válidas para correlaciones (valores negativos o faltantes).")

pairs_to_plot = []
if top_pairs_log:
    pairs_to_plot = top_pairs_log[:5]
elif top_pairs_raw:
    pairs_to_plot = top_pairs_raw[:5]
if pairs_to_plot:
    print("\nScatter de los pares principales:")
    if top_pairs_log:
        df_for_scatter = df_log
    else:
        df_for_scatter = df_raw
    for (a, b), score in pairs_to_plot:
        scatter_robusto(df_for_scatter, a, b, title=f"{a} vs {b}  (|ρ|={score:.2f})")
else:
    print("\nNo hay pares suficientes para graficar.")
